<a href="https://colab.research.google.com/github/Lutris98/API_Cheatsheets/blob/main/Sklearn_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pandas ver1.2.1<br>
Sklearn ver0.24.1

#1.EDA

■ Pandas(with Numpy) 

In [ ]:
import pandas as pd

•불러오기

In [ ]:
titanic_df=pd.read_csv(’C:\\Users\\john\\titanic_train.csv’) #csv는 ,로 구별한 파일양식 
from google.colab import drive
drive.mount('content/gdrive')
titanic_df=pd.read_csv('/content/gdrive/Mydrive/Dataset/titanic_train.csv')

•만들기

In [ ]:
df=pd.DataFrame(ndarray,columns=['col1','col2']) 
ndarray=df.values #ndarray가 필요한 작업들이 있다(특히 columns series를 for루프에 넣을 리스트로 만들때)

•Numpy(list약간확장)

In [ ]:
ndarray=np.array([1,2,3]) #list랑 약간 다름 #np.arange(10), np.zeros((3,2))로 편하게 생성가능
array2=array1.reshape(2,3) #차원변경 #-1넣으면 호환되는 값을 대신 대입

In [ ]:
sort_indices=np.argsort(score_array) 
print(name_array[sort_indices]) #메타데이터가 없는 ndarray에서 활용도 높음

In [ ]:
np.dot(A,B) 
np.transpose(A)

##1)Basic statistics

In [ ]:
titanic_df.head(3) #상위 세건 직접 보기
titanic_df.info() #shape,dtype,Null개수 
titanic_df.describe() #기초통계량

##2)Dropping&Creating features

In [ ]:
titanic_df.drop(['Age','Sex'],axis=1,inplace=True) #axis필수,웬만하면 inplace

In [ ]:
titanic_df['New_feature']=[1,2,3] #칼럼 추가
titanic_df['Family']=titianic_df['SibSp']*10+titanic_df['Parch']
titanic_df['Name_len']=titanic_df['Name'].apply(lambda:len(x)) #len(titanic['Name])이 아니라 apply lambda

•불린 인덱싱

In [ ]:
titanic_boolean=titanic_df[(titanic_df['Age']>60)&(titanic_df['Sex']=='female')] #새로운 dataframe만들때
df.loc[(df.Age>15)&(df.Age<=30), 'Age_band']=2 #새로운 feature만들때

##3)Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

#2.Preprocessing(with Visualization)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig,ax=plt.subplots(figsize=(16,8),ncols=4,nrows=2) #figsize로 figure객체의 픽셀조절 #subplot은 위치값인 axs도
features=['RM','ZN','INDUS']
for i,feature in enumerate(features):
    row=int(i/4) #각 figure배치는 직접 ax보고 확인
    col=i%4
    sns.regplot(data=bostonDF,X=feature,y='PRICE',ax=axs[row][col]) #간단한 회귀계산에 그래프까지


##1)Null filling

In [ ]:
titanic_df['Cabin']=titanic_df['Cabin'].fillna(0)
titanic_df['Cabin'].isna().sum() #확인

##2)Scaling&Outlier removal

##3)Dimension reduction

•PCA<br>
공분산행렬(symmetric)을 EVD, 고유값 첫번째, 두번째큰 고유벡터로 분해 

•Truncated SVD<br> 
PCA와 다르게 rectangular로 확장<br>
∑최소로 diagonal하게 만들고 U,Vt도 맞춰주기

•LDA<br>
클래스간 분산최대, 클래스내 분산최소

#3.Modeling

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV, KFold, StratififedKFold #Stratified는 label분포가 치중되었을때(주로 분류)

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(titanic_feature,titanic_label,test_size=0.2,random_state=0) #X가 먼저 #재현을 위해 randomstate필수

In [ ]:
kfold=KFold(n_splits=5,random_state=0) #5개의 kfold객체 생성 
for train_index,test_index in kfold.split(titanic_feature,titanic_label): #각 인덱스를 ndarray로 반환 
  X_train,X_test=titanic_feature[train_index], titanic_feature[test_index]
  y_train,y_test=titanic_label[train_index], titanic_label[test_index]
  lgbm_clf.fit(X_train,y_train)
  cv_accuracy.append(accuracy_score(y_test,lgbm.predict(X_test)))
  print('cv정확도:',cv_accuracy)
print('\n평균cv정확도:',np.mean(cv_accuracy))

■ 분류

• RandomForest(Bagging)<br>
병렬적으로 결정트리에 bootstrapping된 데이터 학습

•XGBoost(Boosting)<br> 
GBM대비 병렬학습가능해 빠르고 과적합규제/가지치기로 split재검토,내부적 교차검증,결손값처리,조기중단

•LightGBM(Boosting)<br> 
XGBoost와 달리 leafwise 분할(시간,공간복잡도 개선, 성능비슷)

•LogisticRegression<br>
sigmoid함수반환값을 확률로 간주

•Metric<br>
confusion matrix에서 FP와 FN줄이기

■ 회귀

•Scaling<br>
Feature,Target을 로그변환

•PolynomialFeatures<br>
파생변수추가해 설명력향상 but overfit 위험

•Regulation<br>

•Metric<br>
MSE, R2

■ 군집화

•Kmeans<br>

•MeansShift<br>

•DBSCAN<br>

•Metric<br>
silhouette